In [1]:
pip install ioc-fanger

In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [19]:
import re
import requests
import PyPDF2
import ioc_fanger
from urllib.parse import urlparse

In [4]:
def extract_text_from_pdf(pdf_file):
    text = ''
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [5]:
pdf = '/content/Shining a Light on DARKSIDE Ransomware Operations _ Google Cloud Blog.pdf'

In [8]:
pdfcontent = extract_text_from_pdf(pdf)

In [9]:
def fetch_url_content(url):
    response = requests.get(url)
    return response.text

In [10]:
webcontent = fetch_url_content('https://cloud.google.com/blog/topics/threat-intelligence/shining-a-light-on-darkside-ransomware-operations/')

In [18]:
def extract_custom_urls(text):
    pattern = r'\b(hxxps?://\S+?)(?:</?[^>]+>|(?=\s|$))'
    matches = re.findall(pattern, text, re.IGNORECASE)
    cleaned_matches = [match.rstrip('.') for match in matches]
    return cleaned_matches

In [12]:
defangediocs = extract_custom_urls(webcontent)
print(defangediocs)

['hxxps://104.193.252[.]197:443/', 'hxxps://162.244.81[.]253:443/', 'hxxps://185.180.197[.]86:443/', 'hxxps://athaliaoriginals[.]com/', 'hxxps://lagrom[.]com:443/font.html', 'hxxps://lagrom[.]com:443/night.html', 'hxxps://lagrom[.]com:443/online.html', 'hxxps://lagrom[.]com:443/send.html', 'hxxps://lagrom[.]com/find.html?key=id#-', 'hxxps://dl.teamviewer[.]com/download/version_15x/TeamViewer_Setup.exe', 'hxxps://downloads.rclone[.]org/v1.54.0/rclone-v1.54.0-windows-amd64.zip', 'hxxp://sol-doc[.]xyz/sol/ID-482875588', 'hxxps://downloads.rclone[.]org/v1.54.0/rclone-v1.54.0-windows-amd64.zip']


In [13]:
def filter_valid_iocs(defanged_iocs):
    valid_iocs = []
    for ioc in defanged_iocs:
        fanged_ioc = ioc_fanger.fang(ioc)
        valid_iocs.append(fanged_ioc)
    return valid_iocs

In [14]:
alliocs = filter_valid_iocs(defangediocs)
print(alliocs)

['https://104.193.252.197:443/', 'https://162.244.81.253:443/', 'https://185.180.197.86:443/', 'https://athaliaoriginals.com/', 'https://lagrom.com:443/font.html', 'https://lagrom.com:443/night.html', 'https://lagrom.com:443/online.html', 'https://lagrom.com:443/send.html', 'https://lagrom.com/find.html?key=id#-', 'https://dl.teamviewer.com/download/version_15x/TeamViewer_Setup.exe', 'https://downloads.rclone.org/v1.54.0/rclone-v1.54.0-windows-amd64.zip', 'http://sol-doc.xyz/sol/ID-482875588', 'https://downloads.rclone.org/v1.54.0/rclone-v1.54.0-windows-amd64.zip']


In [20]:
def separate_iocs(iocs):
    separated_iocs = []
    for ioc in iocs:
        parsed_url = urlparse(ioc.strip("'"))

        scheme = parsed_url.scheme
        domain = parsed_url.hostname
        ip_address = parsed_url.hostname
        port = parsed_url.port
        path = parsed_url.path

        separated_iocs.append({
            'original_ioc': ioc.strip("'"),
            'scheme': scheme,
            'domain': domain,
            'ip_address': ip_address,
            'port': port,
            'path': path
        })

    return separated_iocs


In [22]:
categorized_iocs = separate_iocs(alliocs)

for ioc in categorized_iocs:
    print("Original IOC:", ioc['original_ioc'])
    print("Scheme:", ioc['scheme'])
    print("Domain:", ioc['domain'])
    print("IP Address:", ioc['ip_address'])
    print("Port:", ioc['port'])
    print("Path:", ioc['path'])
    print()

Original IOC: https://104.193.252.197:443/
Scheme: https
Domain: 104.193.252.197
IP Address: 104.193.252.197
Port: 443
Path: /

Original IOC: https://162.244.81.253:443/
Scheme: https
Domain: 162.244.81.253
IP Address: 162.244.81.253
Port: 443
Path: /

Original IOC: https://185.180.197.86:443/
Scheme: https
Domain: 185.180.197.86
IP Address: 185.180.197.86
Port: 443
Path: /

Original IOC: https://athaliaoriginals.com/
Scheme: https
Domain: athaliaoriginals.com
IP Address: athaliaoriginals.com
Port: None
Path: /

Original IOC: https://lagrom.com:443/font.html
Scheme: https
Domain: lagrom.com
IP Address: lagrom.com
Port: 443
Path: /font.html

Original IOC: https://lagrom.com:443/night.html
Scheme: https
Domain: lagrom.com
IP Address: lagrom.com
Port: 443
Path: /night.html

Original IOC: https://lagrom.com:443/online.html
Scheme: https
Domain: lagrom.com
IP Address: lagrom.com
Port: 443
Path: /online.html

Original IOC: https://lagrom.com:443/send.html
Scheme: https
Domain: lagrom.com
IP 

In [23]:
categorized_iocs

[{'original_ioc': 'https://104.193.252.197:443/',
  'scheme': 'https',
  'domain': '104.193.252.197',
  'ip_address': '104.193.252.197',
  'port': 443,
  'path': '/'},
 {'original_ioc': 'https://162.244.81.253:443/',
  'scheme': 'https',
  'domain': '162.244.81.253',
  'ip_address': '162.244.81.253',
  'port': 443,
  'path': '/'},
 {'original_ioc': 'https://185.180.197.86:443/',
  'scheme': 'https',
  'domain': '185.180.197.86',
  'ip_address': '185.180.197.86',
  'port': 443,
  'path': '/'},
 {'original_ioc': 'https://athaliaoriginals.com/',
  'scheme': 'https',
  'domain': 'athaliaoriginals.com',
  'ip_address': 'athaliaoriginals.com',
  'port': None,
  'path': '/'},
 {'original_ioc': 'https://lagrom.com:443/font.html',
  'scheme': 'https',
  'domain': 'lagrom.com',
  'ip_address': 'lagrom.com',
  'port': 443,
  'path': '/font.html'},
 {'original_ioc': 'https://lagrom.com:443/night.html',
  'scheme': 'https',
  'domain': 'lagrom.com',
  'ip_address': 'lagrom.com',
  'port': 443,
  'p